In [1]:
## import necessary libs
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.functional as F
import torchvision.utils as vutils
from torch.utils.data.sampler import SubsetRandomSampler

from vanilla_resnet import DACNN_RES,SmallBlock


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

In [2]:
batch_size = 64
valid_size = 0.2

train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    
])

test_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    
])

# choose the training and test datasets
train_data = torchvision.datasets.CIFAR10('data', train=True,
                              download=True, transform=train_transform)
test_data = torchvision.datasets.CIFAR10('data', train=False,
                             download=True, transform=test_transform)

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

print(len(train_data),len(test_data))

Files already downloaded and verified
Files already downloaded and verified
50000 10000


In [3]:
model = DACNN_RES()
model.to(device)

DACNN_RES(
  (conv_1): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn_1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_global): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (sections): Sequential(
    (0): Sequential(
      (0): SmallBlock(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): SmallBlock(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum

In [4]:
def accuracy(loader,model):
    correct = 0
    model.eval()
    for data,labels in loader:
        data,labels = data.to(device),labels.to(device)
        output = model(data)
        _,pred = torch.max(output,1) ## get the predictions
        correct_tensor = pred.eq(labels.data.view_as(pred))
        correct  += sum(np.squeeze(correct_tensor.cpu().numpy()))
    return correct/len(loader.sampler)

In [5]:
def acc_by_class(loader,model):
    ## calculate per-class accuracy
    correct = [0]*10
    total = [0]*10
    model.eval()
    for data,labels in loader:
        data,labels = data.to(device),labels.to(device)
        output = model(data)
        _,pred = torch.max(output,1) ## get the predictions
        correct_tensor = pred.eq(labels.data.view_as(pred))
        
        correct_np = correct_tensor.cpu().numpy().squeeze()
        labels = labels.cpu().numpy().squeeze()
        
        for pred,label in zip(correct_np,labels):
            correct[label] += pred
            total[label] += 1 ## count total no of examples
    
    return correct,total

In [6]:
def target_list(loader,model):
    ## calculate per-class accuracy
    pred_list = []
    true_list = []
    model.eval()
    for data,labels in loader:
        data,labels = data.to(device),labels.to(device)
        output = model(data)
        _,pred = torch.max(output,1) ## get the predictions
        pred_np = pred.detach().cpu().numpy().squeeze()
        labels = labels.cpu().numpy().squeeze()
        
        pred_list.extend(pred_np)
        true_list.extend(labels)
        
    return np.asarray(pred_list),np.asarray(true_list)

In [7]:
optimizer = torch.optim.Adagrad(model.parameters(),lr=1e-1)
criterion = nn.CrossEntropyLoss()

In [8]:
## Training Loop
n_epochs = 90
valid_acc_max = -np.Inf
train_losses = []
valid_losses = []
val_acc = []

for epoch in range(1,n_epochs+1): 
    
    train_loss = 0.0 ## running losses
    valid_loss = 0.0
    
    model.train() ## training mode
    print('Part 1 Started')
    for data,labels in train_loader:
        
        data,labels = data.to(device),labels.to(device)
        batch_size = data.size(0)
        
        optimizer.zero_grad() ## clear the gradient
        output = model(data) ## get the output
        loss = criterion(output,labels) ## get the output
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()*batch_size
    
    model.eval()
    print('Part 2 Started')
    for data,labels in valid_loader:
        
        data,labels = data.to(device),labels.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output,labels)
        valid_loss += loss.item()*batch_size    
    
    
    train_loss /= len(train_loader.sampler)
    valid_loss /= len(valid_loader.sampler)
    
    
    valid_acc = accuracy(valid_loader,model)
    
    valid_losses.append(valid_loss)
    train_losses.append(train_loss)
    
    val_acc.append(valid_acc)
    
        
    print('Epoch : {} \tTraining Loss : {} \tValidation Loss :{} \tValidation Acc : {}'.format(epoch,train_loss,valid_loss,valid_acc))
    print('-'*100)
    if valid_acc >= valid_acc_max:
        print('Validation Acc. increased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_acc_max,
        valid_acc))
        torch.save(model.state_dict(), './vanilla_resnet.pt')
        valid_acc_max = valid_acc

Part 1 Started
Part 2 Started
Epoch : 1 	Training Loss : 1.6851723158836365 	Validation Loss :1.3744869247436524 	Validation Acc : 0.5093
----------------------------------------------------------------------------------------------------
Validation Acc. increased (-inf --> 0.509300).  Saving model ...
Part 1 Started
Part 2 Started
Epoch : 2 	Training Loss : 1.0602142126083374 	Validation Loss :0.9638714912414551 	Validation Acc : 0.6607
----------------------------------------------------------------------------------------------------
Validation Acc. increased (0.509300 --> 0.660700).  Saving model ...
Part 1 Started
Part 2 Started
Epoch : 3 	Training Loss : 0.8828075497627258 	Validation Loss :0.9240636863708496 	Validation Acc : 0.6764
----------------------------------------------------------------------------------------------------
Validation Acc. increased (0.660700 --> 0.676400).  Saving model ...
Part 1 Started
Part 2 Started
Epoch : 4 	Training Loss : 0.768561780166626 	Vali

Part 2 Started
Epoch : 30 	Training Loss : 0.17835342462062836 	Validation Loss :0.6869901029586792 	Validation Acc : 0.8119
----------------------------------------------------------------------------------------------------
Validation Acc. increased (0.811600 --> 0.811900).  Saving model ...
Part 1 Started
Part 2 Started
Epoch : 31 	Training Loss : 0.17014710170030595 	Validation Loss :0.7141573665618897 	Validation Acc : 0.8076
----------------------------------------------------------------------------------------------------
Part 1 Started
Part 2 Started
Epoch : 32 	Training Loss : 0.16121800878047943 	Validation Loss :0.726026975440979 	Validation Acc : 0.8076
----------------------------------------------------------------------------------------------------
Part 1 Started
Part 2 Started
Epoch : 33 	Training Loss : 0.1531729877948761 	Validation Loss :0.7350792072296143 	Validation Acc : 0.8116
-------------------------------------------------------------------------------------

Part 2 Started
Epoch : 63 	Training Loss : 0.046679657363891604 	Validation Loss :1.0452733249664308 	Validation Acc : 0.8075
----------------------------------------------------------------------------------------------------
Part 1 Started
Part 2 Started
Epoch : 64 	Training Loss : 0.041168723952770235 	Validation Loss :1.0496207653045655 	Validation Acc : 0.808
----------------------------------------------------------------------------------------------------
Part 1 Started
Part 2 Started
Epoch : 65 	Training Loss : 0.040719001698493955 	Validation Loss :1.0285468217849731 	Validation Acc : 0.8099
----------------------------------------------------------------------------------------------------
Part 1 Started
Part 2 Started
Epoch : 66 	Training Loss : 0.03809108040332794 	Validation Loss :1.0631483386993408 	Validation Acc : 0.806
----------------------------------------------------------------------------------------------------
Part 1 Started
Part 2 Started
Epoch : 67 	Training

In [9]:
model.load_state_dict(torch.load('./vanilla_resnet.pt')) ## Load the best model

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [10]:
correct,total = acc_by_class(valid_loader,model)
print(correct)
print(total)
acc  = [i/j for i,j in zip(correct,total)]
print(acc)
sum(correct)/sum(total)

[858, 906, 742, 666, 784, 689, 849, 851, 883, 920]
[1024, 1014, 967, 1027, 972, 990, 960, 1036, 989, 1021]
[0.837890625, 0.893491124260355, 0.7673216132368149, 0.6484907497565725, 0.8065843621399177, 0.695959595959596, 0.884375, 0.8214285714285714, 0.8928210313447927, 0.901077375122429]


0.8148

In [11]:
correct,total = acc_by_class(test_loader,model)
print(correct)
print(total)
acc  = [i/j for i,j in zip(correct,total)]
print(acc)
sum(correct)/sum(total)

[838, 901, 754, 649, 807, 752, 888, 824, 878, 890]
[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
[0.838, 0.901, 0.754, 0.649, 0.807, 0.752, 0.888, 0.824, 0.878, 0.89]


0.8181

In [12]:
correct,total = acc_by_class(_loader,model)
print(correct)
print(total)
acc  = [i/j for i,j in zip(correct,total)]
print(acc)
sum(correct)/sum(total)

NameError: name '_loader' is not defined